In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.stats.power as smp
from statsmodels.sandbox.stats.multicomp import multipletests

In [2]:
df = pd.DataFrame.from_csv("salaries.csv")
df.describe()

,Unnamed: 0.1,Agency,Salary2010,Salary2011,Salary2012,Salary2013
count,52256.000000,52256.000000,5.225600e+04,5.225600e+04,5.225600e+04,5.225600e+04
mean,156430.169205,409.339655,4.938067e+04,5.212639e+04,5.251105e+04,5.030586e+04
std,89673.627737,155.483410,3.391431e+04,3.443960e+04,3.523870e+04,3.672471e+04
min,10.000000,11.000000,1.000000e+02,1.000000e+02,1.020000e+02,1.010000e+02
25%,83462.750000,305.000000,3.043250e+04,3.379500e+04,3.392975e+04,3.065450e+04
50%,155251.000000,360.000000,4.632200e+04,4.818700e+04,4.888950e+04,4.709300e+04
75%,225747.500000,490.000000,6.422325e+04,6.604500e+04,6.633725e+04,6.564025e+04
max,339762.000000,696.000000,1.982918e+06,2.529168e+06,2.736431e+06,2.633642e+06


# Задание 1

#### Одновыборочный параметрический тест для Salary2010

Рассмотрим правостороннюю гипотезу для поля Salary2010:

$$H_0: \mu = \mu_0,$$
$$H_1: \mu > \mu_0.$$
$$\alpha = 0.05 $$

In [3]:
mu0 = 49300

In [4]:
ppf = stats.t.ppf(0.95, df['Salary2010'].count()-1)

In [5]:
t_obs = (df['Salary2010'].mean() - mu0)/(df['Salary2010'].std()/np.sqrt(df['Salary2010'].count()))
print "ppf =", ppf
print "t_obs =", t_obs
if abs(t_obs) < ppf:
    print "Принимаем нулевую гипотезу"
else:
    print "Принимаем альтернативную гипотезу"

ppf = 1.64488278773
t_obs = 0.54372873691
Принимаем нулевую гипотезу


#### Двухвыборочный параметрический тест для Salary2010 и Salary 2011

Нулевая гипотеза и t-статистика:
$$ H_0: \mu_1 = \mu_2.$$
$$ H_1: \mu_1 \neq \mu_2.$$
$$ \alpha = 0.05 $$
$$ t_{obs} = \frac{\bar{X}_1 - \bar{X}_2}{\sqrt{s^2_{X_1}+s^2_{X_2}}\sqrt{\frac{1}{n}}}. $$

In [6]:
ppf = stats.t.ppf(0.95, df['Salary2010'].count()+df['Salary2011'].count()-1)

In [7]:
t_obs = (df['Salary2010'].mean() - df['Salary2011'].mean()) / \
                (np.sqrt(stats.tvar(df['Salary2010'].values) + stats.tvar(df['Salary2011'].values)) * \
                np.sqrt(1.0/len(df)))
print "ppf =", ppf
print "t_obs =", t_obs
if abs(t_obs) < ppf:
    print "Принимаем нулевую гипотезу"
else:
    print "Принимаем альтернативную гипотезу"

ppf = 1.64486820707
t_obs = -12.9856341717
Принимаем альтернативную гипотезу


In [8]:
pvalues = []

In [9]:
t, p = stats.ttest_ind(df['Salary2010'],df['Salary2011'], equal_var=True)
pvalues.append(p)

__В следующих тестах гипотезы можно считать аналогичными__

#### Двухвыборочный непараметрический тест для Salary2010 и Salary 2011 (Wilcoxon rank-sum statistic)

In [10]:
t, p = stats.ranksums(df['Salary2010'], df['Salary2011'])
print "t =", t
print "p value =", p
if abs(t) < ppf:
    print "Принимаем нулевую гипотезу"
else:
    print "Принимаем альтернативную гипотезу"
pvalues.append(p)

t = -15.3047883038
p value = 7.1032737434e-53
Принимаем альтернативную гипотезу


#### Параметрический тест для парных наблюдений


In [11]:
sample1 = np.random.choice(df['Salary2010'],5000)
sample2 = np.random.choice(df['Salary2010'],5000)

In [12]:
stats.ttest_rel(sample1, sample2)

Ttest_relResult(statistic=0.66734122361484138, pvalue=0.50458500341499934)

#### Непараметрический тест для парных наблюдений (Flinger test)

In [13]:
stats.fligner(sample1, sample2)

FlignerResult(statistic=1.1848774181017394, pvalue=0.27636528235493985)

# Задание 2

Параметрический тест с точной альтернативной гипотезой для Salary2010:

$$H_0: \mu = \mu_0,$$
$$H_1: \mu \neq \mu_0.$$
$$\alpha = 0.05 $$

Мощность теста:

In [14]:
smp.ttest_power(df['Salary2010'].mean()/df['Salary2010'].std(), nobs= df['Salary2010'].count(), alpha=0.05, alternative='two-sided')

1.0

# Задание 3

In [15]:
hyp, new_p, sid, bonf = multipletests(pvalues, alpha=0.05, method='bonferroni')
print "Hypothisis rejected: ", hyp
print "corrected p_values:", new_p

Hypothisis rejected:  [ True  True]
corrected p_values: [  3.16231358e-38   1.42065475e-52]


#### Видно, что результаты не изменились, нулевая гипотеза отвергается в обоих случаях

# Задание 4

#### Доверительный интервал для среднего

In [16]:
# Confidence
conf = 0.95

m, se = df['Salary2010'].mean(), df['Salary2010'].sem()
h = se * stats.t.ppf((1+conf)/2., len(df)-1)
print "Mean:", m
print "Confidence interval for mean:", [m-h, m+h]

Mean: 49380.6672344
Confidence interval for mean: [49089.88154364247, 49671.452925126672]


#### Доверительный интервал для медианы:

In [17]:
def bootstrap(arr, n):
    indexes = np.floor(np.random.rand(n)*len(arr)).astype(int)
    sample = arr[indexes]
    return sample

In [18]:
m = df['Salary2010'].mean()
errors = []
n=100
for i in xrange(n):
    errors.append(m - bootstrap(df['Salary2010'], len(df)).median())

In [19]:
# Confidence
conf = 0.95

h = np.percentile(np.sort(errors), conf*100)
df['Salary2010'].median()
print "Median:", m
print "Confidence interval for median:", [m-h, m+h]

Median: 49380.6672344
Confidence interval for median: [46509.775000000001, 52251.559468769141]
